In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

from __future__ import print_function
import numpy as np # linear algebra
import torch
import torch.nn as nn
import torch.nn.functional as F

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

[]


In [2]:
class Net(nn.Module):
    
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16*5*5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), 2)
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
    def num_flat_features(self, x):
        size = x.size()[1:]
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

nnet = Net()
print(nnet)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [3]:
params = list(nnet.parameters())
print(len(params))
print(params[0].size())

10
torch.Size([6, 1, 5, 5])


In [4]:
ip = torch.randn(1,1,32,32)
out = nnet(ip)
print(out)

tensor([[-0.0740,  0.0035,  0.0601,  0.0759, -0.0920, -0.0768,  0.0578,  0.1079,
         -0.1034,  0.0776]], grad_fn=<AddmmBackward>)


In [5]:
nnet.zero_grad()
out.backward(torch.randn(1,10))

In [6]:
output = nnet(ip)
target = torch.randn(10)
target = target.view(1,-1)
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

tensor(0.6007, grad_fn=<MseLossBackward>)


In [7]:
nnet.zero_grad()

print("conv1.bias.grad before backward")
print(nnet.conv1.bias.grad)

loss.backward()

print("conv2.bias.grad after backward")
print(nnet.conv1.bias.grad)

conv1.bias.grad before backward
tensor([0., 0., 0., 0., 0., 0.])
conv2.bias.grad after backward
tensor([-0.0034,  0.0091, -0.0032, -0.0019,  0.0080, -0.0093])


In [8]:
learning_rate = 0.01
for f in nnet.parameters():
    f.data.sub_(f.grad.data * learning_rate)   #weight update rule

In [9]:
import torch.optim as optim

optimizer = optim.SGD(nnet.parameters(), lr=0.01)
optimizer.zero_grad()

#training
output = nnet(ip)
loss = criterion(output, target)
loss.backward()
optimizer.step()